In [1]:
pip install langchain langchain-openai pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# 1. Define structured output
class Task(BaseModel):
    task_name: str = Field(description="Name of the task.")
    estimated_days: int = Field(description="Estimated days to complete the task.")
    team_member: str = Field(description="Assigned team member's name.")

class ProjectPlan(BaseModel):
    project_description: str = Field(description="Overall description of the project.")
    tasks: list[Task] = Field(description="List of detailed tasks for the project.")
    risks: list[str] = Field(description="Potential risks associated with the project plan.")
    steps:list[str] = Field(description= "Steps to Write Better AI Prompts for Project Manager.")
    phase_details: list[str] = Field(description="In Depth details for this project phase.")
# 2. Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

# 3. Define prompt
parser = PydanticOutputParser(pydantic_object=ProjectPlan)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class project manager assistant. Your goal is to create a structured project plan in JSON format. {format_instructions}"),
    ("human", "Create a project plan for the following project: {project_goal}")
]).partial(format_instructions=parser.get_format_instructions())

# 4. Build chain
planning_chain = prompt | llm | parser

# 5. Run chain
project_goal_input = input("Enter a subject: ")

#project_goal_input = "Develop a basic e-commerce website using Python and Django."
#print(f"Generating project plan for: '{project_goal_input}'...\n")

response = planning_chain.invoke({"project_goal": project_goal_input})

# 6. Process output
print("--- Generated Project Plan ---")
print(f"Project Description: {response.project_description}\n")

print("Tasks:")
for i, task in enumerate(response.tasks):
    print(f"* {i+1}. {task.task_name} ({task.estimated_days} days) - Assigned to: {task.team_member}")

print("\nRisks:")
for i, risk in enumerate(response.risks):
    print(f"* {i+1}. {risk}")

print("\nSteps for Project Manager:")
for i, step in enumerate(response.steps):
    print(f"* {i+1}. {step}")
    
print("Project Phase:")
for i, phase in enumerate(response.phase_details):
    print(f"* {i+1}. {phase}")




Enter a subject:  Develop a basic e-commerce website using Python and Django.


--- Generated Project Plan ---
Project Description: Develop a basic e-commerce website using Python and Django.

Tasks:
* 1. Set up Django project (2 days) - Assigned to: John
* 2. Design database schema (3 days) - Assigned to: Alice
* 3. Create models for products and users (2 days) - Assigned to: Bob
* 4. Implement user authentication (3 days) - Assigned to: Alice
* 5. Develop product listing page (4 days) - Assigned to: John
* 6. Implement shopping cart functionality (3 days) - Assigned to: Bob
* 7. Design and implement checkout process (5 days) - Assigned to: Alice

Risks:
* 1. Potential delays due to third-party dependencies
* 2. Security vulnerabilities in the e-commerce website

Steps for Project Manager:
* 1. Gather requirements for the e-commerce website
* 2. Set up development environment with Python and Django
* 3. Create wireframes and design mockups
* 4. Start development with basic features
* 5. Perform testing and debugging
* 6. Deploy the e-commerce website
Project Phas